In [1]:
import pandas as pd
import polars as pl
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.0f}'.format)

In [2]:
def mic_equity_geog(df_in, equity_group):

    df_inc = df.pivot_table(
        index='person_work_mic',
        columns=equity_group,
        values='psexpfac',
        aggfunc='sum'
    ).reset_index()

    df_inc = df_inc.rename_axis(None, axis=1)
    pd.set_option('display.float_format', '{:,.0f}'.format)
    df_inc['Percent in Equity Geog'] = df_inc[1] / (df_inc[0] + df_inc[1])
    df_inc['Percent in Equity Geog'] = df_inc['Percent in Equity Geog'].fillna(0)
    df_inc['Percent in Equity Geog'] = (df_inc['Percent in Equity Geog'] * 100).round(1).astype(str) + '%'
    df_inc.sort_values(by=0, ascending=False)

    df_inc.rename(columns={'person_work_mic': 'MIC Work Location',
                        0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population'}, inplace=True)
    df_inc.index = df_inc['MIC Work Location']
    df_inc.drop('MIC Work Location', axis=1, inplace=True)
    # Percent of people of color
    
    return df_inc

# Workers in Manufacturing-Industrial Centers (MICs)

In [3]:
df = pd.read_csv(r'../../../../outputs/agg/dash/mic_workers.csv')

In [4]:
df = pd.read_csv(r'../../../../outputs/agg/dash/mic_workers.csv')
df = df[df['pwtyp'].isin(['Paid Full-Time Worker', 'Paid Part-Time Worker'])]
df.loc[df['person_work_mic'].isnull(), 'person_work_mic'] = 'Outside MIC'
_df = df[['psexpfac','person_work_mic']].groupby('person_work_mic').sum()[['psexpfac']]
_df.index
_df.rename(columns={'psexpfac': 'Total Workers'}, inplace=True)
_df


,Total Workers
person_work_mic,
Ballard-Interbay,"15,759"
Cascade,"8,818"
Duwamish,"57,713"
Frederickson,"4,070"
Kent MIC,"42,361"
North Tukwila,"6,353"
Outside MIC,"2,083,702"
Paine Field / Boeing Everett,"35,436"
Port of Tacoma,"9,592"


In [5]:
mic_equity_geog(df, "hh_efa_pov200").loc[_df.index]

,Below Regional Average,Above Regional Average,Higher Share of Equity Population,Percent in Equity Geog
person_work_mic,,,,
Ballard-Interbay,"11,487","2,824","1,448",19.7%
Cascade,"5,119","2,581","1,118",33.5%
Duwamish,"34,572","14,785","8,356",30.0%
Frederickson,"2,121","1,416",533,40.0%
Kent MIC,"21,237","12,650","8,474",37.3%
North Tukwila,"3,153","1,970","1,230",38.5%
Outside MIC,"1,298,601","521,554","263,547",28.7%
Paine Field / Boeing Everett,"20,464","9,623","5,349",32.0%
Port of Tacoma,"4,349","3,242","2,001",42.7%


In [6]:
mic_equity_geog(df, "hh_efa_poc").loc[_df.index]

,Below Regional Average,Above Regional Average,Higher Share of Equity Population,Percent in Equity Geog
person_work_mic,,,,
Ballard-Interbay,"10,006","3,817","1,936",27.6%
Cascade,"7,762",915,141,10.5%
Duwamish,"26,114","16,534","15,065",38.8%
Frederickson,"2,257","1,251",562,35.7%
Kent MIC,"14,757","13,086","14,518",47.0%
North Tukwila,"2,258","1,796","2,299",44.3%
Outside MIC,"1,124,284","609,885","349,533",35.2%
Paine Field / Boeing Everett,"19,969","12,348","3,119",38.2%
Port of Tacoma,"4,625","3,136","1,831",40.4%


In [7]:
mic_equity_geog(df, "hh_efa_lep").loc[_df.index]

,Below Regional Average,Above Regional Average,Higher Share of Equity Population,Percent in Equity Geog
person_work_mic,,,,
Ballard-Interbay,"10,928","2,770","2,061",20.2%
Cascade,"7,482",959,377,11.4%
Duwamish,"30,704","12,648","14,361",29.2%
Frederickson,"3,163",571,336,15.3%
Kent MIC,"17,476","10,868","14,017",38.3%
North Tukwila,"2,700","1,533","2,120",36.2%
Outside MIC,"1,298,486","440,063","345,153",25.3%
Paine Field / Boeing Everett,"18,994","8,368","8,074",30.6%
Port of Tacoma,"6,333","1,924","1,335",23.3%


# Commute Characteristics

In [8]:
df = pd.read_csv(r'../../../../outputs/agg/dash/tour_mic_dest.csv')
# Commute tours to MICs
df = df[df['pdpurp']=="Work"]
df.loc[df['tour_d_mic'].isnull(), 'tour_d_mic'] = 'Outside MIC'

# tmodetp by tour_d_mic
df_mode = pd.pivot_table(
    df,
    index='tour_d_mic',
    columns='tmodetp',
    values='toexpfac',
    aggfunc='sum'
)

# mode share by tour_d_mic
df_mode = df_mode.div(df_mode.sum(axis=1), axis=0)
df_mode = df_mode.fillna(0)
df_mode = df_mode.reset_index()
df_mode.rename(columns={'tour_d_mic': 'MIC Work Location'}, inplace=True)
df_mode.index = df_mode['MIC Work Location']
df_mode.drop('MIC Work Location', axis=1, inplace=True)

df_mode['Transit'] = df_mode['Transit']+df_mode['Park']
df_mode['HOV'] = df_mode['HOV2'] + df_mode['HOV3+']
df_mode['Walk/Bike/Other'] =  df_mode['Walk']+df_mode['Bike']+df_mode['TNC']
# df_mode = df_mode.sort_values(by='Drive Alone', ascending=False)
# Show results as percentages
df_mode = df_mode.map(lambda x: f"{x:.1%}")
# df_mode.sort_values(by='Walk', ascending=False, inplace=True)


df_mode[['SOV', 'HOV', 'Transit', 'Walk/Bike/Other']]

tmodetp,SOV,HOV,Transit,Walk/Bike/Other
MIC Work Location,,,,
Ballard-Interbay,65.8%,25.2%,1.6%,7.4%
Cascade,66.6%,30.6%,0.1%,2.7%
Duwamish,65.6%,28.6%,2.9%,2.9%
Frederickson,67.5%,30.7%,0.0%,1.9%
Kent MIC,66.0%,31.1%,1.2%,1.6%
North Tukwila,67.1%,29.5%,1.5%,1.9%
Outside MIC,61.8%,27.6%,3.5%,7.2%
Paine Field / Boeing Everett,67.8%,29.9%,0.3%,2.0%
Port of Tacoma,67.3%,30.8%,0.6%,1.3%


In [9]:
# Commute distance
pd.set_option('display.float_format', '{:,.1f}'.format)

df = pd.read_csv(r'../../../../outputs/agg/dash/tour_distance_mic.csv')
df = df[df['pdpurp'] == 'Work']
df['wt_dist'] = df['tautodist_bin'] * df['toexpfac']
df = df.groupby("person_work_mic").agg(
    {'wt_dist': 'sum', 'toexpfac': 'sum'}
)
df['average_distance'] = df['wt_dist']/df['toexpfac']
df[['average_distance']]

,average_distance
person_work_mic,
Ballard-Interbay,10.1
Cascade,10.6
Duwamish,12.7
Frederickson,8.8
Kent MIC,12.5
North Tukwila,12.6
Paine Field / Boeing Everett,11.1
Port of Tacoma,11.9
Puget Sound Industrial Center- Bremerton,13.7


# Demographics

In [10]:
# Demographics 
df = pd.read_csv(r'../../../../outputs/agg/dash/mic_workers.csv')
df['income_wt'] = df['hhincome_thousands'] * df['psexpfac']
df['income_wt'] = df['income_wt'].fillna(0)
df = df.groupby('person_work_mic').agg(
    {'psexpfac': 'sum', 'income_wt': 'sum'}
).reset_index()
df["avg_weighted_hh_income"] = df['income_wt']/df['psexpfac']
df['avg_weighted_hh_income'] = df['avg_weighted_hh_income'].apply(lambda x: f"${x:,.0f}")
df[['person_work_mic', 'avg_weighted_hh_income']]

,person_work_mic,avg_weighted_hh_income
0,Ballard-Interbay,"$188,095"
1,Cascade,"$150,416"
2,Duwamish,"$184,490"
3,Frederickson,"$143,607"
4,Kent MIC,"$164,145"
5,North Tukwila,"$163,192"
6,Paine Field / Boeing Everett,"$157,184"
7,Port of Tacoma,"$145,910"
8,Puget Sound Industrial Center- Bremerton,"$154,080"
9,Sumner Pacific,"$149,225"


In [11]:
df = pd.read_csv(r'../../../../outputs/agg/dash/mic_workers.csv')
df['hhsize_wt'] = df['hhsize'] * df['psexpfac']
df['hhsize_wt'] = df['hhsize_wt'].fillna(0)
df = df.groupby('person_work_mic').agg(
    {'psexpfac': 'sum', 'hhsize_wt': 'sum'}
)
df['avg_wt_hhsize'] = df['hhsize_wt']/df['psexpfac']
df = df.reset_index()
df[['person_work_mic', 'avg_wt_hhsize']]

,person_work_mic,avg_wt_hhsize
0,Ballard-Interbay,2.9
1,Cascade,3.5
2,Duwamish,3.1
3,Frederickson,3.6
4,Kent MIC,3.4
5,North Tukwila,3.2
6,Paine Field / Boeing Everett,3.2
7,Port of Tacoma,3.4
8,Puget Sound Industrial Center- Bremerton,3.2
9,Sumner Pacific,3.4


In [12]:
df = pd.read_csv(r'../../../../outputs/agg/dash/mic_workers.csv')
# Pivot table of worker type by person_work_mic
df_worker_type = df.pivot_table(
    index='person_work_mic',
    columns='pwtyp',
    values='psexpfac',
    aggfunc='sum'
)

# Share by worker type

df_worker_type = df_worker_type.div(df_worker_type.sum(axis=1), axis=0)
df_worker_type.rename(columns={'Paid Full-Time Worker': 'Full Time', 'Paid Part-Time Worker': 'Part Time'})
df_worker_type.map(lambda x: f"{x:.1%}")

pwtyp,Paid Full-Time Worker,Paid Part-Time Worker
person_work_mic,,
Ballard-Interbay,82.8%,17.2%
Cascade,73.7%,26.3%
Duwamish,87.1%,12.9%
Frederickson,69.2%,30.8%
Kent MIC,85.3%,14.7%
North Tukwila,80.3%,19.7%
Paine Field / Boeing Everett,83.5%,16.5%
Port of Tacoma,84.5%,15.5%
Puget Sound Industrial Center- Bremerton,64.1%,35.9%
